<a href="https://colab.research.google.com/github/shayanAbbasi1995/BGSE-Macro-Finance/blob/main/Edgar_SEC_Scraping_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Get data

## Libraries

In [ ]:
# import our libraries
import requests
import pandas as pd
from bs4 import BeautifulSoup
import ast
import time
from random import seed
from random import random
import requests
from requests.exceptions import ConnectionError

##Connect to Google Drive


In [ ]:
# Run this if running in Google Collab
# Mount google drive if running from Google Collab
from google.colab import drive
drive.mount('/content/drive')

# Set current directory if running from Google Collab

import os
os.chdir('/content/drive/My Drive/Colab Notebooks/edgar/SEC_Filing')# here use your path to current notebook

Mounted at /content/drive


## Read tickers and CIK

In [ ]:
symbs = pd.read_stata("../Firm-code-info.dta")
cik_firstlink_df = pd.read_csv('first_link_df.csv')

In [ ]:
primary_links = cik_firstlink_df[cik_firstlink_df.astype(str)['first_links'] != '[]']
primary_links.first_links = primary_links.first_links.apply(lambda x: ast.literal_eval(x))
primary_links.to_csv('primary_links.csv')

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [ ]:
# How many tickers do we have?
print(f' we have {len(symbs.tic.unique())} tickers, {len(symbs.cik.unique())} CIKs, {len(symbs.gvkey.unique())} GVKEYs, {len(symbs.conm.unique())} company names')

 we have 41671 tickers, 30033 CIKs, 41692 GVKEYs, 41689 company names


So, it's best to use tickers or GVKEYs

# Scraping functions and Error handling

In [ ]:
!pip install requests-random-user-agent

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 47 kB 3.9 MB/s 


# RUN

In [ ]:
import requests
import requests_random_user_agent # pip install requests-random-user-agent
import pandas as pd
from bs4 import BeautifulSoup # pip install beautifulsoup4
import re
import sqlite3
from sqlite3 import Error
import os
import sys
from contextlib import closing # pip install contextlib2
import time
from dateutil import parser # pip install python-dateutil
from datetime import datetime
from urllib.request import urlopen
import time
import os
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# how many firms? Look at the number of rows  at the bottom of the table
# However, many of the firms are aquaired or went private and data for them is None 
primary_links

,Unnamed: 0,cik,first_links
4,4,1750.0,[https://www.sec.gov/Archives/edgar/data/1750/...
6,6,313368.0,[https://www.sec.gov/Archives/edgar/data/31336...
10,10,61478.0,[https://www.sec.gov/Archives/edgar/data/61478...
14,14,4911.0,[https://www.sec.gov/Archives/edgar/data/4911/...
18,18,319126.0,[https://www.sec.gov/Archives/edgar/data/31912...
...,...,...,...
30025,30025,921114.0,[https://www.sec.gov/Archives/edgar/data/92111...
30027,30027,1640147.0,[https://www.sec.gov/Archives/edgar/data/16401...
30028,30028,1695295.0,[https://www.sec.gov/Archives/edgar/data/16952...
30029,30029,1822250.0,[https://www.sec.gov/Archives/edgar/data/18222...


In [ ]:
# How many 10K reports?
ult = []
for i in primary_links.first_links:
  for j in i:
    ult.append(j)
print(len(ult))

191384


# Functions

In [ ]:
def is_html(url):
  if url[-3:]=='htm' or url[-4:]=='html':
    return(True)
  else:
    print('link-type unrecognized')
    return(False)

def is_txt(url):
  if url[-3:]=='txt':
    return(True)
  else:
    return(False)

In [ ]:
def get_soup(url):
  headers = {"User-Agent": 'abbasi.shayan1995@gmail.com'}
  response = requests.get(url = url, headers=headers)
  soup = BeautifulSoup(response.content, 'html.parser')
  print('Request Successful')
  print(response.url)
  time.sleep(0.1)
  return soup

def get_filing_link(first_url):
  soup = get_soup(first_url)
  doc_table = soup.find('table', class_='tableFile', summary="Document Format Files")
  #print(doc_table)
  hrefs = doc_table.find_all("a", href=True)
  #print(hrefs)
  temp_links = None
  base_url_sec = r"https://www.sec.gov"
  K10_link = base_url_sec + str([a.get('href') for a in hrefs[0:1]][0])
  if is_html(K10_link):
    return(K10_link)
  else:
    for link in hrefs:
      temp_link = base_url_sec + str(link.get('href'))
      if is_txt(temp_link):
        return(temp_link)
      else:
        pass


In [ ]:
# Installing selenium for google colab
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 983 kB 31.0 MB/s 
     |████████████████████████████████| 359 kB 72.1 MB/s 
     |████████████████████████████████| 138 kB 58.8 MB/s 
     |████████████████████████████████| 55 kB 4.4 MB/s 
     |████████████████████████████████| 4.0 MB 44.9 MB/s 
     |████████████████████████████████| 58 kB 1.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.9 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
G

In [ ]:
def get_dynamic_soup(url, web_driver):
  time.sleep(1)
  web_driver.get(url)
  soup = BeautifulSoup(web_driver.page_source, 'html.parser')
  return(soup)

In [ ]:
def get_year(soup):
  selected_tag = soup.findAll(text = re.compile("fiscal year ended"))
  print(selected_tag)

In [ ]:
def check_filing_form(soup):
  pass

def get_form(url):
  if is_html(url):
    return('html')
  elif is_txt(url):
    return('text')
  else:
    print('Link does not refer to HTML or TEXT')
    return(None)

In [ ]:
def fix_neg_numbers(lines):
  line_num = 1
  line_size = len(lines)-1
  while line_num <= line_size:
    if (lines[line_num] == ')' or lines[line_num] == ')1' or
        lines[line_num] == ')2' or lines[line_num] == ')3'
        or lines[line_num] == ')4'):
      lines[line_num-1] = lines[line_num-1] + ')'
      lines.pop(line_num)
      line_size -= 1
      line_num -= 1
    line_num += 1
    if lines[line_num-1] == lines[line_num-2]:
      lines.pop(line_num-1)
      line_size -= 1
      line_num -= 1
  return(lines)

def remove_blank_lines(lines):
  non_empty_lines = [line for line in lines if line.strip() != ""]
  string_without_empty_lines = ""
  for line in non_empty_lines:
      string_without_empty_lines += line + "\n"
  return(string_without_empty_lines)

In [ ]:
def is_soup_large(soup):
  if soup[0:24] == 'IOPub data rate exceeded.':
    return True
  else:
    return False

def is_unicode_err(soup):
  unicode_err_flag=0
  for table in soup.findAll('table'):
    lines = table.text.split("\n")
    lines = remove_blank_lines(lines)
    lines = lines.split('\n')
    if len(lines)<=5:
      continue
    else:
      unicode_err_flag = 1
    lines = fix_neg_numbers(lines)
  if unicode_err_flag==0:
    return True
  else:
    return False

def is_single_dig(string):
  if len(string.strip()) <=1:
    return True
  else:
    return False

def fix_mix_neg_num(table_list, itr_limit, i):
    if re.findall('\(([^)]+)', table_list[i]) != []:
      extract = '(' + re.findall('\(([^)]+)', table_list[i])[0] + ')'
      table_list[i] = table_list[i].replace(extract, "")
      table_list = table_list[:i] + [extract] + table_list[i:]
      itr_limit += 1
      i += 1
    return table_list, itr_limit, i

In [ ]:
def is_number_tryexcept(s):
    """ Returns True is string is a number. """
    try:
        float(s)
        return True
    except ValueError:
        return False

def remove_paran_footnote(string):
  clean_list = re.findall('\(([^)]+)', string)
  if len(clean_list) >= 1:
    for i in range(len(clean_list)):
      if is_number_tryexcept(clean_list[i]):
        return clean_list[i]
  else:
    return 'A string smaller than length 1' # in is_footnote() it returns False

def is_footnote(string):
  if len(string) <= 4 and len(string) >= 3:
    try:
      if len(remove_paran_footnote(string)) == 1:
        return True
      else:
        return False
    except:
      print('')
  return False

def remove_paran(string):
  if is_number_tryexcept(string):
    return float(string)
  else:
    clean_list = re.findall('\(([^)]+)', string)
    if len(clean_list) >= 1:
      if is_number_tryexcept(clean_list[0]):
        return -1 * float(clean_list[0])
      else:
        return string
    else:
      return string

In [ ]:
def num_years(table):
  num_years = 0
  years = list(range(2000, 2023))
  j = 0 
  for i in range(len(table)):
    if is_number_tryexcept(table[i]):
      if int(float(table[i])) in years and ((j==i-1 or j==0) or (i==0 or j==0)): # to make sure that we only count years that come in a row
        num_years += 1
        j = i
  return num_years

def get_years(table):
  year_list = []
  years = list(range(2000, 2023))
  j = 0 
  for i in range(len(table)):
    if is_number_tryexcept(table[i]):
      year_i = int(float(table[i]))
      if (year_i in years) and ((j==i-1 or j==0) or (i==0 or j==0)): # to make sure that we only count years that come in a row
        year_list.append(year_i)
        j = i
  return year_list

def get_last_year(table):
  return max(get_years(table))

def get_target_jump(table):
  years = get_years(table)
  last_year = get_last_year(table)
  i = 0
  for year in years:
    if year != last_year:
      i += 1
    else:
      i += 1
      return i 

def get_scale(table):
  thousand = ['($’s in thousands)', '($ in thousands)', '(in thousands)',
              '($ in thousands, except per share data)', '$ in thousands',
              '(table only in thousands)', '(dollars in thousands except share data)',
              '(table only in thousands except per share data)', '(dollars in thousands)']
  million = ['(In millions)', '(In millions except per share data)', 
             '($ in millions)', '$ in millions', '(dollars in millions)', 
             '(In millions except earnings per share)', '(in millions)',
             '(in millions except per share data)', '(in millions except share data)',
             '(dollars in millions)']
  billion = [] # Found no firm that report in billions
  for item in table:
    if item in thousand:
      return 1000
    if item in million:
      return 1000000
    if item in billion:
      return 1000000000  
  return 1


In [ ]:
def fix_unicode(table_list): #Fixing unicode \u200b and \xa0 along with some minor abbreviations and footnotes removal
  i = 0
  itr_limit = len(table_list) - 1
  while i <= itr_limit:
    table_list[i] = re.sub(r"\u200b", "\n", table_list[i]) #substitute \u200b by \n
    table_list[i] = re.sub(r"\xa0", " ", table_list[i])  #sub \xa0 by space
    table_list[i] = re.sub(r'(?i)(?<=\d)(?=[a-z])|(?<=[a-z])(?=\d)', '  ', table_list[i]).strip()  #putting double space between letters and numbers 
    table_list[i] = re.sub(r"  ", "\n", table_list[i])
    table_list[i] = table_list[i].replace(":", "")
    table_list[i] = table_list[i].replace(",", "")
    table_list[i] = table_list[i].replace("$", "")
    table_list[i] = table_list[i].replace("%", " % ")
    table_list[i] = table_list[i].replace('(loss)', "")
    table_list[i] = table_list[i].replace('(a)', "")
    table_list[i] = table_list[i].replace('(b)', "")
    table_list[i] = table_list[i].replace('(c)', "")
    table_list[i] = table_list[i].replace('(d)', "")
    table_list[i] = table_list[i].replace('(e)', "")
    table_list[i] = table_list[i].lower().strip()
    i += 1
  return table_list

def fix_paran(table_list):
  i = 0
  itr_limit = len(table_list) - 1
  while i <= itr_limit:
    table_list[i] = table_list[i].strip()
    table_list, itr_limit, i = fix_mix_neg_num(table_list, itr_limit, i)
    i += 1
  return table_list

def remove_singles(table_list): #removing footnotes and irrelevent single digit data
  for item in table_list:
    if len(item)<=2 or is_footnote(item):
      table_list.remove(item)
  return table_list

In [ ]:
def get_tables(soup):
  tables = []
  for table in soup.findAll('table'):
    lines = table.text.split("\n")
    lines = fix_unicode(lines)
    lines = [word for line in lines for word in line.split('\n')]
    lines = remove_blank_lines(lines)
    lines = lines.split('\n')
    if len(lines)<=5: 
      continue
    lines = fix_paran(lines)
    lines = fix_neg_numbers(lines)
    lines = remove_singles(lines)
    #print(lines) #used this in the coding process as a testing feature
    tables.append(lines)
  return tables

In [ ]:
table_dict = {
'sales_key' : ['net sales', 'sales', 'total revenues', 'operating revenues', 
             'revenues', 'total revenue'],

'cosgs_key' : ['cost of goods sold', 'costs and operating expenses', 'cost of sales',
             'operating expenses', 'cost of revenues', 'total costs and expenses'],

'xsga_key' : ['selling and administrative expenses', 'selling general and administrative',
            'selling and administration expense', 'selling general and administrative expenses',
            'selling, general and administrative expenses', 'selling general & admin.',
            'general and administrative', 'selling general and administrative costs',
            'selling general and administrative costs'],

'ppent_key' : ['property plant and equipment net',  'property and equipment', 
             'property plant & equipment net', 'total property plant and equipment net',
             'property and equipment net', 'property and equipment net '], 
             

#second item has a number for net and one for gross
'ppegt_key' : ['property plant and equipment gross', 'property plant and equipment at cost', 
             'property and equipment', 'total property plant and equipment'],

'ppegt_key_exception' : ['less accumulated depreciation', 'accumulated depreciation',
                       'less accumulated depreciation depletion and amortization',
                       'property and equipment net ', 'depreciation and amortization',
                       'depreciation depletion and amortization'], 
# sometimes I have to reduce 'Less accumulated depreciation'

'intang_key' : ['goodwill', 'intangible assets – finite life net', 
              'intangible assets – indefinite life', 'goodwill and intangible assets net',
              'intangibles', 'goodwill and intangibles', 'intangible assets', 
              'intangible assets net', 'intangible assets net '],

'dividend_key' : ['earnings per share',  
                'earnings per share – diluted', 'earnings  per diluted share from continuing',
                'basic-as reported', 'diluted-as reported', 'net earnings per share',
                'diluted  income per common share', 'diluted net (losses) earnings per common share',
                'earnings per common share', 'dividends per common share'],

#Maybe part
'RnD_key' : ['research and development', 'research and development expenses'] 
}

'''
# ***for exact tablles:***
    #1. The search should happen in lower_case(), strip() and () removed

    #2. Check sales not to be 100 if so, remove the table

    #3. check again for single numbers

# ***For similarity tables:***
    #1. check sales and cost of products not having %

    #2. 'Sales', 'net' / 'Cost of', 'products' / 'Selling general', 'and administrative'. look at report 29 - 1: there should be an if for seprated items in tables

    #3. Sometimes the number is sticked to the item. Make sure to fix it: 'Diluted net (losses) earnings per common share192',  Sometimes in year ended we have this: 'Year Ended December 31201920182017' (first item)

    # it can be checked by the number of years and number of available int after the item in table


# total approach: 
    #1. First find the ones with easy access
    #2. Save the ones with problems, go through them and try the exceptions to get the data
    #3. Match the data with competition-related violation dataset
    #4. Run a fixed effect model: see the results / check the assumptions / Look for IV
    #5. Use other models like DiD / Probit, Logit
'''


In [ ]:
def is_similar(item, key_table):
  for key in key_table:
    counter = 0
    item_list = word_tokenize(item) 
    key_list = word_tokenize(key)
    for word in item_list:
      if word in key_list:
        counter += 1
    if counter/len(key) >= 0.6:
      return True
    else:
      return False

def get_similar_table(table, key_table):
  for item in table:
    if is_similar(item.lower().strip(), key_table):
      return table
  return []

def get_exact_table(table, key_table):
  for item in table:
      if item.lower().strip() in key_table:
        return table
  return []

def get_table(table_list, table_dict, key): #SALES, NET SALES, TOTAL SALES ++++++
  target_tables = []
  similar_tables = []
  for table in table_list:
    founded_table = []
    if len(get_years(table))>=2:
      founded_table = get_exact_table(table, table_dict[key])
    if len(founded_table) != 0:
      target_tables.append(founded_table)
    if len(target_tables) == 0:
      founded_table = get_similar_table(table, table_dict[key])
    if len(founded_table) != 0:
      similar_tables.append(founded_table)
  return target_tables, similar_tables

def all_years_first(target_tables):
  if len(target_tables) >= 1:
    for i in range(len(target_tables)):
      years = get_years(target_tables[i])
      j = 0
      while j < len(target_tables[i]):
        try:
          if int(target_tables[i][j]) in years:
            target_tables[i].remove(target_tables[i][j])
            j -= 1
        except:
          pass
        j += 1
      target_tables[i] = years + target_tables[i]
  return target_tables


def iter_target_tables(table_list, table_dict, target_dict, similar_dict):
  for key in table_dict.keys():
    target_tables = []
    similar_tables = []
    target_tables, similar_table = get_table(table_list, table_dict, key)
    #target_tables = all_years_first(target_tables)
    #similar_table = all_years_first(similar_table)
    target_dict[key] = target_tables
    similar_dict[key] = similar_table
  return target_dict, similar_dict

In [ ]:
def cond_nested_table(i, table):
  if i < len(table):
    if is_number_tryexcept(remove_paran(table[i])):
      return True
  else:
    return False

def nested_get_value(table, i, scale):
  shift = num_years(table)
  if shift == 0:
    return 'years not recognized' 
  else:
    years_list = get_years(table)
    if len(years_list) == 0:
      return 'years list 0' 
    if max(years_list) == years_list[0]:
      if cond_nested_table(i+2+shift, table):
        return float(remove_paran(table[i+2+shift])) * scale
      elif cond_nested_table(i+3+(2*shift), table):
        return float(remove_paran(table[i+3+(2*shift)])) * scale
      elif cond_nested_table(i+4+(3*shift), table):
        return float(remove_paran(table[i+4+(3*shift)])) * scale
      elif cond_nested_table(i+5+(4*shift), table):
        return float(remove_paran(table[i+5+(4*shift)])) * scale
      elif cond_nested_table(i+6+(5*shift), table):
        return float(remove_paran(table[i+6+(5*shift)])) * scale
      elif cond_nested_table(i+7+(6*shift), table):
        return float(remove_paran(table[i+7+(6*shift)])) * scale
      elif cond_nested_table(i+8+(7*shift), table):
        return float(remove_paran(table[i+8+(7*shift)])) * scale
    elif max(years_list) == years_list[-1]:
      if cond_nested_table(i+2+shift, table):
        return float(remove_paran(table[i+1+(2*shift)])) * scale
      elif cond_nested_table(i+3+(2*shift), table):
        return float(remove_paran(table[i+2+(3*shift)])) * scale
      elif cond_nested_table(i+4+(3*shift), table):
        return float(remove_paran(table[i+3+(4*shift)])) * scale          
      elif cond_nested_table(i+5+(4*shift), table):
        return float(remove_paran(table[i+4+(5*shift)])) * scale
      elif cond_nested_table(i+6+(5*shift), table):
        return float(remove_paran(table[i+5+(6*shift)])) * scale
      elif cond_nested_table(i+7+(6*shift), table):
        return float(remove_paran(table[i+6+(7*shift)])) * scale
      elif cond_nested_table(i+8+(7*shift), table):
        return float(remove_paran(table[i+7+(8*shift)])) * scale
      
def get_value(table, table_dict, key):
  scale = get_scale(table)
  if key == 'dividend_key':
    scale = 1
  i = 0
  while i < len(table):
    if table[i].strip() in table_dict[key]:
      str_paran_removed = remove_paran(table[i+1])
      if is_number_tryexcept(str_paran_removed) and len(str(str_paran_removed)) >= 2: # Making sure again that the number after is not a footnote number or text
        if int(float(str_paran_removed)) != 100 and int(float(str_paran_removed)) not in get_years(table): # Making sure that the table is not a percentage represented table
           return float(str_paran_removed) * scale
        else:
          return 'nested values'
      i += 1
    else:
      i += 1
  return 'nested values'


def iter_nested_table(table, table_dict, key):
  scale = get_scale(table)
  if key == 'dividend_key':
    scale = 1
  value = None
  i = 0
  while i < len(table):
    if table[i] in table_dict[key]:
      value = nested_get_value(table, i, scale)
      if value != 'not even nested' or value  != 'No value found':
        return value
    i += 1
  else:
    return('No value found')

def extract_vars(target_dict, similar_dict, table_dict):
  value = None
  d = {'year': None, 'sales_key' : None, 'cosgs_key' : None, 'xsga_key' : None, 'ppent_key' : None, 
       'ppegt_key' : None, 'ppegt_key_exception' : None, 'intang_key' : None,
       'dividend_key' : None, 'RnD_key' : None}
  keys = target_dict.keys()
  for key in keys:
    if len(target_dict[key]) != 0:
      for table in target_dict[key]:
        try:
          if key == 'sales_key':
            d['year'] = max(get_years(table))
        except:
          pass
        got_key = 0
        value = get_value(table, table_dict, key)
        if value != 'nested values':
          d[key] = value
          got_key = 1
          break
        if got_key == 0 and value == 'nested values':
          if key == 'dividend_keys':
            value = float(remove_paran(table[i+2]))
            d[key] = value
          value = iter_nested_table(table, table_dict, key)
          if value != 'No value found' and key != 'dividend_keys':
            d[key] = value
            break
          else:
            d[key] = None
  print(d)
  return d

In [ ]:
target_dict = {'sales_key' : [], 'cosgs_key' : [], 'xsga_key' : [], 'ppent_key' : [], 
               'ppegt_key' : [], 'ppegt_key_exception' : [], 'intang_key' : [],
               'dividend_key' : [], 'RnD_key' : []}

similar_dict = {'sales_key' : [], 'cosgs_key' : [], 'xsga_key' : [], 'ppent_key' : [], 
               'ppegt_key' : [], 'ppegt_key_exception' : [], 'intang_key' : [],
               'dividend_key' : [], 'RnD_key' : []}

In [ ]:
def get_var_dict(wd, primary_links, target_dict, similar_dict, i, j):
  try:
    link = primary_links.first_links.iloc[i][j]
    url = get_filing_link(link)
    soup = get_dynamic_soup(url, wd)
    tables = get_tables(soup)
    target_dict, similar_dict = iter_target_tables(tables, table_dict, target_dict, similar_dict) 
    dic = extract_vars(target_dict, similar_dict, table_dict)
    dic['cik'] = primary_links.cik.iloc[i]
    dic['i'] = i
    dic['j'] = j
    return dic
  except:
    return 'MaxRetryError'


In [ ]:
def get_wd(): 
  seed(42)
  value = int(random() * 10000)
  user_agent = f'abbasi.shayan{value}@gmail.com'
  chrome_options.add_argument(f"user-agent={user_agent}")
  wd = webdriver.Chrome('chromedriver', chrome_options=chrome_options)
  return wd

In [ ]:
def add_new_row(new_dic_row, dataset):
  new_row = pd.DataFrame(new_dic_row, index=[0])
  dataset = pd.concat([new_row, dataset.loc[:]]).reset_index(drop=True)
  return dataset

In [ ]:
def make_dataset(i, limit, primary_links, dataset, wd, j=0, target_dict=target_dict, similar_dict=similar_dict):
  try:
    while i != limit:
      for k in range(j, len(primary_links.first_links.iloc[i])):
        print(f'i = {i}, j = {k} of limit = {limit} and time periods = {len(primary_links.first_links.iloc[i])}')
        new_dic_row = get_var_dict(wd, primary_links, target_dict, similar_dict, i, k)
        dataset = add_new_row(new_dic_row, dataset)
      i += 1
    return dataset, i, k, True
  except:
    print('Error in getting data/connnection')
    return dataset, i, k, False

In [ ]:
def main(i, limit, primary_links, target_dict=target_dict, similar_dict=similar_dict):
  dataset = pd.DataFrame()
  j = 0
  while True:
    time.sleep(60)
    wd = get_wd()
    dataset, i, j, cond = make_dataset(i, limit, primary_links, dataset, wd, j)
    print(f' i = {i} of limit = {limit} ------------->RECONNECT')
    if cond:
      print(f'limit = {limit}, i = {i} --------------> DONE')
      return dataset

In [ ]:
# Remember, the txt files (reports from before 2001/2000) are not going to be retrieved, thus it will return None values

begin = 220
finish = 230
end = begin + 10
while begin != finish:
  dataset = main(begin, end, primary_links)
  dataset.to_csv(f'dataset_{begin}_{end}.csv')
  begin += 10
  end = begin+10

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  


i = 220, j = 0 of limit = 230 and time periods = 29
Request Successful
https://www.sec.gov/Archives/edgar/data/8146/000119312521114283/0001193125-21-114283-index.htm
{'year': None, 'sales_key': None, 'cosgs_key': None, 'xsga_key': None, 'ppent_key': None, 'ppegt_key': None, 'ppegt_key_exception': None, 'intang_key': None, 'dividend_key': None, 'RnD_key': None}
i = 220, j = 1 of limit = 230 and time periods = 29
Request Successful
https://www.sec.gov/Archives/edgar/data/8146/000119312520103834/0001193125-20-103834-index.htm

{'year': None, 'sales_key': None, 'cosgs_key': 46325.0, 'xsga_key': 11357.0, 'ppent_key': 11268.0, 'ppegt_key': None, 'ppegt_key_exception': 6284.0, 'intang_key': 12034.0, 'dividend_key': None, 'RnD_key': 8084.0}
i = 220, j = 2 of limit = 230 and time periods = 29
Request Successful
https://www.sec.gov/Archives/edgar/data/8146/000119312519102788/0001193125-19-102788-index.htm

{'year': None, 'sales_key': None, 'cosgs_key': 45279.0, 'xsga_key': 11123.0, 'ppent_key': 